# <div style="text-align: center;"> DATA SCIENCE CAPSTONE PROJECT </div>

## <div style="text-align: center;"> DOMINATE THE TRANSFER MARKET: FOOTBALL PLAYER PRICE PREDICTION </div>

### <div style="text-align: center;"> DATA CRAWLING SECTION </div>

#### 0. How did we create the final Dataframe?

In [ ]:
PATH_TO_PLAYERS_LINK = "C:/Users/Admin/OneDrive - Hanoi University of Science and Technology/Desktop/players_link.csv"
PATH_TO_PLAYERS_DATA = "C:/Users/Admin/OneDrive - Hanoi University of Science and Technology/Desktop/players_data.csv"
PATH_TO_NOT_GK_PLAYERS_DATA = "C:/Users/Admin/OneDrive - Hanoi University of Science and Technology/Desktop/not_gk_players_data.csv"
PATH_TO_GK_PLAYERS_DATA = "C:/Users/Admin/OneDrive - Hanoi University of Science and Technology/Desktop/gk_players_data.csv"
PATH_TO_NOT_GK_PLAYERS_LINK = "C:/Users/Admin/OneDrive - Hanoi University of Science and Technology/Desktop/not_gk_players_link.csv"
PATH_TO_GK_PLAYERS_LINK = "C:/Users/Admin/OneDrive - Hanoi University of Science and Technology/Desktop/gk_players_link.csv"
PATH_TO_NOT_GK_PLAYERS_STAT = "C:/Users/Admin/OneDrive - Hanoi University of Science and Technology/Desktop/not_gk_players_stat.csv"
PATH_TO_GK_PLAYERS_STAT = "C:/Users/Admin/OneDrive - Hanoi University of Science and Technology/Desktop/gk_players_stat.csv"
PATH_TO_NOT_GK_PLAYERS = 'C:/Users/Admin/OneDrive - Hanoi University of Science and Technology/Desktop/not_gk_players.csv'
PATH_TO_GK_PLAYERS = 'C:/Users/Admin/OneDrive - Hanoi University of Science and Technology/Desktop/gk_players.csv'

Listing vital features for valuing players is our priority first by approaching some related papers. After researching topics and surfing the web page Transfermarkt, we noticed that to get all the features needed, we have to reach two different addresses linked to each players, which is called **(general) data** and **(performance) statistics**. While the former directs us quite a general information about players such as the name, club, age, ... which could be effortlessly remembered for all football viewers, the latter focus on players' detailed analysising statistics in the match that is more about the expertise. To approach both tasks, we must get the players'id which is their distinctive features in Transfermarkt. The only consideration is that in the **(performance) statistics** work, the **goalkeeper** and **other positions** have two different set of evaluation, requiring us split the collected id into two new files. After having all the needed feature obtained, our last job is to concatenate all the Dataframe together.

#### 1. IMPORTING LIBRARIES

In [2]:
import re
import requests, time
import pandas as pd
from bs4 import BeautifulSoup

#### 2. PLAYERS' LINK SCRAPING

Getting the browser's user agent for sending requests

In [11]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36", "accept-language": "en-US,en;q=0.9"}

Getting a list of club needed scraping (top 5 football leagues in the world are considered)

In [12]:
country = pd.DataFrame([{'CountryID': 189, 'Country': 'England'},
        {'CountryID': 40, 'Country': 'Germany'},
        {'CountryID': 75, 'Country': 'Italy'},
        {'CountryID': 50, 'Country': 'France'},
        {'CountryID': 157, 'Country': 'Spain'}])

Getting leagues' name and url (top 2 football leagues each countries are considered)

In [5]:
league_name, league_url = [], []
for i in range(len(country)):
    url = f"https://www.transfermarkt.com/wettbewerbe/national/wettbewerbe/{country.loc[i,'CountryID']}"
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    for j in range(1, 3):
        league_span = soup.select('.inline-table a')[j]
        league_name.append(league_span.get('title'))
        league_url.append('https://www.transfermarkt.com' + league_span.get('href') + '/plus/?saison_id=')

Getting Transfermarkt's href of all the players in these leagues (6 seasons are considered)

In [ ]:
All_Players_Link = []
for league, url in zip(league_name, league_url):
    for season in range(2018, 2024):
        page = requests.get(url + str(season), headers = headers)
        soup = BeautifulSoup(page.content, 'html.parser')
        
        club_urls = [link.get("href") for link in soup.select("#yw1 .no-border-links a:nth-child(1)")]
        for club_url in club_urls:
            club_id = club_url.split("/")[-3]
            club_page = requests.get("https://www.transfermarkt.com" + club_url, headers=headers)                  
            soup2 = BeautifulSoup(club_page.content, "html.parser")

            players_list = soup2.select(".inline-table .hauptlink > a")
            All_Players_Link.extend(p.get('href') for p in players_list)

Dropping duplicates and store them to a csv file named "players_link" serving as players web's adress to scrape their datas

In [ ]:
pd.DataFrame(All_Players_Link).drop_duplicates().to_csv(PATH_TO_PLAYERS_LINK, index = False)

#### 3. PLAYERS' (GENERAL) INFORMATION SCRAPING

Defining function *scraping_player_data* to scrape players' (general) data

In [2]:
def scraping_players_data(url, players_datas):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36", "accept-language": "en-US,en;q=0.9"}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")
    data = {}

    pattern = r"/(\d+)$"
    match = re.search(pattern, url)
    player_id = match.group(1)
    data["player_id"] = player_id

    try:
        name = soup.select_one('h1[class = "data-header__headline-wrapper"]').text.split("\n")[-1].strip()
    except AttributeError:
        name = None
    data["name"] = name

    try:
        player_club = soup.select_one("span[class = 'data-header__club']").text.strip()
    except AttributeError:
        player_club = None
    except ValueError:
        player_club = None
    except IndexError:
        player_club = None
    data["player_club"] = player_club

    try:
        age = float(soup.select_one('li[class="data-header__label"]').text.split("\n")[-2].split()[-1].strip("()"))
    except AttributeError:
        age = None
    except ValueError:
        age = None
    except IndexError:
        age = None
    data["age"] = age

    try:
        position = soup.find('dd', class_='detail-position__position').text
    except AttributeError:
        position = None
    except ValueError:
        position = None
    except IndexError:
        position = None
    data["position"] = position

    try:
        market_value = soup.select_one('a[class="data-header__market-value-wrapper"]').text.split(" ")[0].replace('€', '')
        if "m" in market_value:
            market_value = market_value.replace("m", "")
            market_value = float(market_value)*1000
        elif "k" in market_value:
            market_value = market_value.replace("k", "")
            market_value = float(market_value)
    except AttributeError:
        market_value = None
    except ValueError:
        market_value = None
    except IndexError:
        market_value = None
    data["market_value"] = market_value
    
    try:
        nationality = soup.find('span', itemprop = "nationality").text.strip()
    except AttributeError:
        nationality = None
    except ValueError:
        nationality = None
    except IndexError:
        nationality = None
    data["nationality"] = nationality

    try:
        player_height = float(re.search("Height:.*?([0-9].*?)\n", soup.text, re.DOTALL).group(1).strip().split(" ")[0].replace(",", "."))
    except AttributeError:
        player_height = None
    except ValueError:
        player_height = None
    except IndexError:
        player_height = None
    data["player_height"] = player_height

    try:
        player_agent = re.search("Agent:.*?([A-z].*?)\n", soup.text, re.DOTALL).group(1).strip()
    except AttributeError:
        player_agent = None
    except ValueError:
        player_agent = None
    except IndexError:
        player_agent = None
    data["player_agent"] = player_agent

    try:
        strong_foot = soup.select('span[class = "info-table__content info-table__content--bold"]')[6].text
    except AttributeError:
        strong_foot = None
    except ValueError:
        strong_foot = None
    except IndexError:
        strong_foot = None
    data["strong_foot"] = strong_foot

    try:
        contract_value_time = float(re.search("Contract expires: (.*)", soup.text).group(1).split()[-1])
    except AttributeError:
        contract_value_time = None
    except ValueError:
        contract_value_time = None
    data["contract_value_time"] = contract_value_time

    return data

Defining column *data_column* corresponding the data scraped by the function *scraping_players_data*. This step is to create the Dataframe *players_data*

Note: Dataframe *players_data* is not only used as a partial Dataframe (which would then be concatenated with other Dataframe to create a final Dataframe 
for predicting model) but also called to determine players' position. We will then use this position to create two new Dataframes for evaluating players' performance
as the estimation criteria is different for **goalkeeper** and **other positions** (based on Transfermarkt's data)

In [3]:
data_column = ["player_id", "name", "player_club", "age", "position", "market_value", "nationality", "player_height", "player_agent", "strong_foot", "contract_value_time"]

Creating Dataframe *players_data*

In [5]:
players_data = pd.DataFrame(columns = data_column).astype(str)
hyperlink = 'https://www.transfermarkt.com' + pd.read_csv(PATH_TO_PLAYERS_LINK)

for i in range(len(hyperlink)):
    single_player_data = scraping_players_data(hyperlink.loc[i, "0"], players_data)
    players_data = pd.concat([players_data, pd.DataFrame([single_player_data])], ignore_index=True)

pd.DataFrame(players_data).to_csv(PATH_TO_PLAYERS_DATA, index=False)

#### 4. PLAYERS' (PERFORMANCE) STATISTICS SCRAPING

Creating two new csv basis for **goalkeeper** and **other positions**

In [2]:
players_data = pd.read_csv(PATH_TO_PLAYERS_DATA)

not_gk_players_data = players_data[players_data['position'] != 'Goalkeeper']
gk_players_data = players_data[players_data['position'] == 'Goalkeeper']

not_gk_players_data.to_csv(PATH_TO_NOT_GK_PLAYERS_DATA, index=False)
gk_players_data.to_csv(PATH_TO_GK_PLAYERS_DATA, index=False)

Splitting the csv file *players_link* into two new csv file for **goalkeeper** link and **other positions**.

In [ ]:
gk_players_data = pd.read_csv(PATH_TO_GK_PLAYERS_DATA)
players_link = pd.read_csv(PATH_TO_PLAYERS_LINK)

gk_ids = gk_players_data['player_id'].astype(str)

not_gk_players_link = players_link[~players_link.iloc[:, 0].str.split('/').str[-1].isin(gk_ids)]
gk_players_link = players_link[players_link.iloc[:, 0].str.split('/').str[-1].isin(gk_ids)]

not_gk_players_link.to_csv(PATH_TO_NOT_GK_PLAYERS_LINK, index=False)
gk_players_link.to_csv(PATH_TO_GK_PLAYERS_LINK, index=False)

Defining function *scraping_not_gk_stat* to scrape **other players**' (performance) statistics

In [3]:
def scraping_not_gk_stat(url, name):
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36", "accept-language": "en-US,en;q=0.9"}

    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")
    not_gk_stat = {}

    try:
        appearances = soup.find_all("td", {"class": "zentriert"})[1].text
        if (appearances == "-"):
            appearances = 0
        else:
            appearances = float(appearances)
    except ValueError:
        appearances = None
    except AttributeError:
        appearances = None
    except IndexError:
        appearances = None
    not_gk_stat["appearances"] = appearances

    try:
        PPG = soup.find_all("td", {"class": "zentriert"})[2].text
        if (PPG == "-"):
            PPG = 0
        else:
            PPG = float(PPG)
    except ValueError:
        PPG = None
    except AttributeError:
        PPG = None
    except IndexError:
        PPG = None
    not_gk_stat["PPG"] = PPG 

    try:
        goals = soup.find_all("td", {"class": "zentriert"})[3].text
        if (goals == "-"):
            goals = 0
        else:
            goals = float(goals)
    except ValueError:
        goals = None
    except AttributeError:
        goals = None
    except IndexError:
        goals = None
    not_gk_stat["goals"] = goals

    try:
        assists = soup.find_all("td", {"class": "zentriert"})[4].text
        if (assists == "-"):
            assists = 0
        else:
            assists = float(assists)
    except ValueError:
        assists = None
    except AttributeError:
        assists = None
    except IndexError:
        assists = None
    not_gk_stat["assists"] = assists

    try:
        own_goals = soup.find_all("td", {"class": "zentriert"})[5].text
        if (own_goals == "-"):
            own_goals = 0
        else:
            own_goals = float(own_goals)
    except ValueError:
        own_goals = None
    except AttributeError:
        own_goals = None
    except IndexError:
        own_goals = None
    not_gk_stat["own_goals"] = own_goals 

    try:
        substitutions_on = soup.find_all("td", {"class": "zentriert"})[6].text
        if (substitutions_on == "-"):
            substitutions_on = 0
        else:
            substitutions_on = float(substitutions_on)
    except ValueError:
        substitutions_on = None
    except AttributeError:
        substitutions_on = None
    except IndexError:
        substitutions_on = None
    not_gk_stat["substitutions_on"] = substitutions_on 

    try:
        substitutions_off = soup.find_all("td", {"class": "zentriert"})[7].text
        if (substitutions_off == "-"):
            substitutions_off = 0
        else:
            substitutions_off = float(substitutions_off)
    except ValueError:
        substitutions_off = None
    except AttributeError:
        substitutions_off = None
    except IndexError:
        substitutions_off = None
    not_gk_stat["substitutions_off"] = substitutions_off
    
    try:
        yellow_cards = soup.find_all("td", {"class": "zentriert"})[8].text
        if (yellow_cards == "-"):
            yellow_cards = 0
        else:
            yellow_cards = float(yellow_cards)
    except ValueError:
        yellow_cards = None
    except AttributeError:
        yellow_cards = None
    except IndexError:
        yellow_cards = None
    not_gk_stat["yellow_cards"] = yellow_cards

    try:
        second_yellow_cards = soup.find_all("td", {"class": "zentriert"})[9].text
        if (second_yellow_cards == "-"):
            second_yellow_cards = 0
        else:
            second_yellow_cards = float(second_yellow_cards)
    except ValueError:
        second_yellow_cards = None
    except AttributeError:
        second_yellow_cards = None
    except IndexError:
        second_yellow_cards = None
    not_gk_stat["second_yellow_cards"] = second_yellow_cards

    try:
        red_cards = soup.find_all("td", {"class": "zentriert"})[10].text
        if (red_cards == "-"):
            red_cards = 0
        else:
            red_cards = float(red_cards)
    except ValueError:
        red_cards = None
    except AttributeError:
        red_cards = None
    except IndexError:
        red_cards = None
    not_gk_stat["red_cards"] = red_cards

    try:
        penalty_goals = soup.find_all("td", {"class": "zentriert"})[11].text
        if (penalty_goals == "-"):
            penalty_goals = 0
        else:
            penalty_goals = float(penalty_goals)
    except ValueError:
        penalty_goals = None
    except AttributeError:
        penalty_goals = None
    except IndexError:
        penalty_goals = None
    not_gk_stat["penalty_goals"] = penalty_goals

    try:
        minutes_per_goal = soup.find_all("td", {"class": "rechts"})[1].text.split("'")[0]
        if (minutes_per_goal == "-"):
            minutes_per_goal = 0
        else:
            minutes_per_goal = float(minutes_per_goal)
    except ValueError:
        minutes_per_goal = None
    except AttributeError:
        minutes_per_goal = None
    except IndexError:
        minutes_per_goal = None
    not_gk_stat["minutes_per_goal"] = minutes_per_goal

    try:
        minutes_played = soup.find_all("td", {"class": "rechts"})[2].text.split("'")[0]
        if (minutes_played == "-"):
            minutes_played = 0
        else:
            minutes_played = float(minutes_played)
    except ValueError:
        minutes_played = None
    except AttributeError:
        minutes_played = None
    except IndexError:
        minutes_played = None
    not_gk_stat["minutes_played"] = minutes_played

    return not_gk_stat

Defining column *not_gk_stat_column* corresponding the data scraped by the function *scraping_not_gk_stat*. This step is to create the Dataframe *not_gk_players_stat*

In [4]:
not_gk_stat_column = ["appearances", "PPG", "goals", "assists", "own_goals", "substitutions_on", "substitutions_off", "yellow_cards", "second_yellow_cards", "red_cards", "penalty_goals", "minutes_per_goal", "minutes_played"]

Creating Dataframe *not_gk_players_stat*

In [ ]:
not_gk_players_stat = pd.DataFrame(columns = not_gk_stat_column).astype(str)
not_gk_players_link = 'https://www.transfermarkt.com' + pd.read_csv(PATH_TO_NOT_GK_PLAYERS_LINK)

for i in range(len(not_gk_players_link)):
    id = not_gk_players_link.loc[i, "0"].split('spieler/')[-1]
    name = not_gk_players_link.loc[i, "0"].split('com/')[-1].split('/profil')[0].replace(' ', '-')
    not_gk_players_hyperlink = f"https://www.transfermarkt.com/{name}/leistungsdatendetails/spieler/{id}/plus/1?saison=2024&verein=&liga=&wettbewerb=&pos=&trainer_id="
    single_not_gk_player_stat = scraping_not_gk_stat(not_gk_players_hyperlink, name)
    not_gk_players_stat = pd.concat([not_gk_players_stat, pd.DataFrame([single_not_gk_player_stat])], ignore_index=True)

pd.DataFrame(not_gk_players_stat).to_csv(PATH_TO_NOT_GK_PLAYERS_STAT, index=False)

Defining function *scraping_gk_stat* to scrape **goalkeeper**' (performance) statistics

In [7]:
def scraping_gk_stat(url, name):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
               "accept-language": "en-US,en;q=0.9"}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")
    gk_stat = {}

    try:
        appearances = soup.find_all("td", {"class": "zentriert"})[1].text
        if (appearances == "-"):
            appearances = 0
        else:
            appearances = float(appearances)
    except ValueError:
        appearances = None
    except AttributeError:
        appearances = None
    except IndexError:
        appearances = None
    gk_stat["appearances"] = appearances

    try:
        PPG = soup.find_all("td", {"class": "zentriert"})[2].text
        if (PPG == "-"):
            PPG = 0
        else:
            PPG = float(PPG)
    except ValueError:
        PPG = None
    except AttributeError:
        PPG = None
    except IndexError:
        PPG = None
    gk_stat["PPG"] = PPG 

    try:
        goals = soup.find_all("td", {"class": "zentriert"})[3].text
        if (goals == "-"):
            goals = 0
        else:
            goals = float(goals)
    except ValueError:
        goals = None
    except AttributeError:
        goals = None
    except IndexError:
        goals = None
    gk_stat["goals"] = goals

    try:
        own_goals = soup.find_all("td", {"class": "zentriert"})[4].text
        if (own_goals == "-"):
            own_goals = 0
        else:
            own_goals = float(own_goals)
    except ValueError:
        own_goals = None
    except AttributeError:
        own_goals = None
    except IndexError:
        own_goals = None
    gk_stat["own_goals"] = own_goals 

    try:
        substitutions_on = soup.find_all("td", {"class": "zentriert"})[5].text
        if (substitutions_on == "-"):
            substitutions_on = 0
        else:
            substitutions_on = float(substitutions_on)
    except ValueError:
        substitutions_on = None
    except AttributeError:
        substitutions_on = None
    except IndexError:
        substitutions_on = None
    gk_stat["substitutions_on"] = substitutions_on 

    try:
        substitutions_off = soup.find_all("td", {"class": "zentriert"})[6].text
        if (substitutions_off == "-"):
            substitutions_off = 0
        else:
            substitutions_off = float(substitutions_off)
    except ValueError:
        substitutions_off = None
    except AttributeError:
        substitutions_off = None
    except IndexError:
        substitutions_off = None
    gk_stat["substitutions_off"] = substitutions_off
    
    try:
        yellow_cards = soup.find_all("td", {"class": "zentriert"})[7].text
        if (yellow_cards == "-"):
            yellow_cards = 0
        else:
            yellow_cards = float(yellow_cards)
    except ValueError:
        yellow_cards = None
    except AttributeError:
        yellow_cards = None
    except IndexError:
        yellow_cards = None
    gk_stat["yellow_cards"] = yellow_cards

    try:
        second_yellow_cards = soup.find_all("td", {"class": "zentriert"})[8].text
        if (second_yellow_cards == "-"):
            second_yellow_cards = 0
        else:
            second_yellow_cards = float(second_yellow_cards)
    except ValueError:
        second_yellow_cards = None
    except AttributeError:
        second_yellow_cards = None
    except IndexError:
        second_yellow_cards = None
    gk_stat["second_yellow_cards"] = second_yellow_cards

    try:
        red_cards = soup.find_all("td", {"class": "zentriert"})[9].text
        if (red_cards == "-"):
            red_cards = 0
        else:
            red_cards = float(red_cards)
    except ValueError:
        red_cards = None
    except AttributeError:
        red_cards = None
    except IndexError:
        red_cards = None
    gk_stat["red_cards"] = red_cards

    try:
        goals_conceded = soup.find_all("td", {"class": "zentriert"})[10].text
        if (goals_conceded == "-"):
            goals_conceded = 0
        else:
            goals_conceded = float(goals_conceded)
    except ValueError:
        goals_conceded = None
    except AttributeError:
        goals_conceded = None
    except IndexError:
        goals_conceded = None
    gk_stat["goals_conceded"] = goals_conceded

    try:
        clean_sheet = soup.find_all("td", {"class": "zentriert"})[11].text
        if (clean_sheet == "-"):
            clean_sheet = 0
        else:
            clean_sheet = float(clean_sheet)
    except ValueError:
        clean_sheet = None
    except AttributeError:
        clean_sheet = None
    except IndexError:
        clean_sheet = None
    gk_stat["clean_sheet"] = clean_sheet

    try:
        minutes_played = soup.find_all("td", {"class": "rechts"})[1].text.split("'")[0]
        if (minutes_played == "-"):
            minutes_played = 0
        else:
            minutes_played = float(minutes_played)
    except ValueError:
        minutes_played = None
    except AttributeError:
        minutes_played = None
    except IndexError:
        minutes_played = None
    gk_stat["minutes_played"] = minutes_played

    return gk_stat

Defining column *gk_stat_column* corresponding the data scraped by the function *scraping_not_gk_stat*. This step is to create the Dataframe *gk_players_stat*

In [8]:
gk_stat_column = ["appearances", "PPG", "goals", "own_goals", "substitutions_on", "substitutions_off", "yellow_cards", "second_yellow_cards", "red_cards", "goals_conceded", "clean_sheet", "minutes_played"]

Creating Dataframe *gk_players_stat*

In [ ]:
gk_players_stat = pd.DataFrame(columns = gk_stat_column).astype(str)
gk_players_link = 'https://www.transfermarkt.com' + pd.read_csv(PATH_TO_NOT_GK_PLAYERS_LINK)

for i in range(len(gk_players_link)):
    id = gk_players_link.loc[i, "0"].split('spieler/')[-1]
    name = gk_players_link.loc[i, "0"].split('com/')[-1].split('/profil')[0].replace(' ', '-')
    gk_players_hyperlink = f"https://www.transfermarkt.com/{name}/leistungsdatendetails/spieler/{id}/plus/1?saison=2024&verein=&liga=&wettbewerb=&pos=&trainer_id="
    single_gk_player_stat = scraping_gk_stat(gk_players_hyperlink, name)
    gk_players_stat = pd.concat([gk_players_stat, pd.DataFrame([single_gk_player_stat])], ignore_index=True)

pd.DataFrame(gk_players_stat).to_csv(PATH_TO_GK_PLAYERS_STAT, index=False)

#### 5. PRE DATA CLEANING

Preparing to concatenate csv file *data* and csv file *stat* together

In [ ]:
not_gk_players_data = pd.read_csv(PATH_TO_NOT_GK_PLAYERS_DATA)
gk_players_data = pd.read_csv(PATH_TO_GK_PLAYERS_DATA)
not_gk_players_stat = pd.read_csv(PATH_TO_NOT_GK_PLAYERS_STAT)
gk_players_stat = pd.read_csv(PATH_TO_GK_PLAYERS_STAT)

Adding the "index" column into every Dataframe serving as the mutual features to concatenate

In [ ]:
not_gk_players_data['Index'] = range(1, len(not_gk_players_data) + 1)
cols = ['Index'] + [col for col in not_gk_players_data.columns if col != 'Index']
not_gk_players_data.to_csv(PATH_TO_NOT_GK_PLAYERS_DATA, index=False)

gk_players_data['Index'] = range(1, len(gk_players_data) + 1)
cols = ['Index'] + [col for col in gk_players_data.columns if col != 'Index']
gk_players_data.to_csv(PATH_TO_GK_PLAYERS_DATA, index=False)

not_gk_players_stat['Index'] = range(1, len(not_gk_players_stat) + 1)
cols = ['Index'] + [col for col in not_gk_players_stat.columns if col != 'Index']
not_gk_players_stat.to_csv(PATH_TO_NOT_GK_PLAYERS_STAT, index=False)

gk_players_stat['Index'] = range(1, len(gk_players_stat) + 1)
cols = ['Index'] + [col for col in gk_players_stat.columns if col != 'Index']
gk_players_stat.to_csv(PATH_TO_GK_PLAYERS_STAT, index=False)

Concatenating csv file *data* and csv file *stat* together

In [ ]:
not_gk_players = pd.merge(not_gk_players_data, not_gk_players_stat, on='Index')
gk_players = pd.merge(gk_players_data, gk_players_stat, on='Index')

not_gk_players.to_csv(PATH_TO_NOT_GK_PLAYERS, index=False)
gk_players.to_csv(PATH_TO_GK_PLAYERS, index=False)

Reading the two newly created csv file

In [ ]:
not_gk_players = pd.read_csv(PATH_TO_NOT_GK_PLAYERS)
gk_players = pd.read_csv(PATH_TO_GK_PLAYERS)

Adjusting columns in the Dataframe *not_gk_players* so that it has the format with the Dataframe *gk_players*

In [ ]:
not_gk_players['goalkeeper_or_not'] = not_gk_players['position'].apply(lambda x: '1' if x == 'Goalkeeper' else '0')
not_gk_players['goals_conceded'] = float(0)
not_gk_players['clean_sheet'] = float(0)
cols = list(not_gk_players.columns)
position_index_4 = cols.index('position')
position_index_5 = cols.index('red_cards')
cols.insert(position_index_4 + 1, cols.pop(cols.index('goalkeeper_or_not')))
cols.insert(position_index_5 + 1, cols.pop(cols.index('goals_conceded')))
cols.insert(position_index_5 + 2, cols.pop(cols.index('clean_sheet')))
not_gk_players = not_gk_players[cols]
not_gk_players.head(1)
not_gk_players.to_csv(PATH_TO_NOT_GK_PLAYERS)
not_gk_players = pd.read_csv(PATH_TO_NOT_GK_PLAYERS)

Adjusting columns in the Dataframe *gk_players* so that it has the format with the Dataframe *not_gk_players*

In [ ]:
gk_players['goalkeeper_or_not'] = gk_players['position'].apply(lambda x: '1' if x == 'Goalkeeper' else '0')
gk_players['assists'] = float(0)
gk_players['penalty_goals'] = float(0)
gk_players['minutes_per_goal'] = float(0)
cols = list(gk_players.columns)
position_index_1 = cols.index('position')
position_index_2 = cols.index('goals')
position_index_3 = cols.index('clean_sheet')
cols.insert(position_index_1 + 1, cols.pop(cols.index('goalkeeper_or_not')))
cols.insert(position_index_2 + 1, cols.pop(cols.index('assists')))
cols.insert(position_index_3 + 1, cols.pop(cols.index('penalty_goals')))
cols.insert(position_index_3 + 2, cols.pop(cols.index('minutes_per_goal')))
gk_players = gk_players[cols]
gk_players.head(1)
gk_players.to_csv(PATH_TO_GK_PLAYERS)
gk_players = pd.read_csv(PATH_TO_GK_PLAYERS)

Concatenating two dataframe *not_gk_players* and *gk_players* together

In [ ]:
final_data = pd.concat([not_gk_players, gk_players])

Some useful function you may need while working with dataframe

In [ ]:
# concatenate two dataframe
""" merged_dataframe = pd.concat([dataframe_a, dataframe_b]) """


# dropping columns of a dataframe
""" dropping_column_dataframe = dataframe.drop(columns=['name_of_column_1', 'name_of_column_2']) """


# swapping two columns of a dataframe
"""columns = list(dataframe.columns)                                                                

name_of_column_1_idx = columns.index('name_of_column_1')
name_of_column_2_idx = columns.index('name_of_column_2')
columns[name_of_column_2_idx], columns[name_of_column_1_idx] = columns[name_of_column_1_idx], columns[name_of_column_2_idx]

dataframe = dataframe[columns]"""


# returning all columns' name
"""column_names = dataframe.columns"""


# returning a column's index
"""column_index = dataframe.columns.get_loc('name_of_the_column')"""


# splitting csv file into a number of small csv files
"""dataframe = pd.read_csv('file's adress.csv')

number_of_rows = len(dataframe)
number_of_row_per_files = number_of_rows // number_of_small_files

for i in range(number_of_small_files):
    start_index = i * number_of_row_per_files
    end_index = (i + 1) * number_of_row_per_files if i < number_of_small_files else number_of_rows
    dataframe_part = dataframe.iloc[start_index:end_index]
    dataframe_part.to_csv(f'file_{i + 1}.csv', index=False)"""


# merging csv files into one csv file
"""dataframe_1 = pd.read_csv('dataframe_1's adress.csv')
dataframe_2 = pd.read_csv('dataframe_2's adress.csv')

merged_df = pd.merge(dataframe_1, dataframe_2, on = 'name_of_the_mutual_column')

merged_df.to_csv('merged_file.csv', index=False)
"""